<a href="https://colab.research.google.com/github/Ignvz/copert_5/blob/main/Factores_emision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#If used from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd "/content/drive/MyDrive/Factores de emision"
!ls

/content/drive/.shortcut-targets-by-id/1rXzOeDHO59D46y_UeQ5BIKirkwBHax7H/Factores_de_emision
'1.A.3.b.i-iv Road Transport Appendix 4 Emission Factors_2020Sept.xlsx'
'Actividad Vehicular.csv'
 BC_emissions.xlsx
 BC_factors.csv
 C5_reducido_prueba.xlsx
 C5_reducido.xlsx
 CH4_emissions.xlsx
 CH4_factors.csv
 CH4_reducido.xlsx
 CO2_emissions.xlsx
 CO2_factors.csv
 CO2_factors.gsheet
 CO2_reducido.xlsx
 CO_emissions.xlsx
 CO_factors.csv
 Copert5.xlsx
 CO_reducido.xlsx
 Euro_7.xlsx
 NOx_emissions.xlsx
 NOx_factors.csv
 NOx_reducido.xlsx
 PM_emissions.xlsx
 PM_factors.csv
 PM_reducido.xlsx
 SOx_emissions.xlsx
 SOx_factors.csv
 Velocidades.csv
 VOC_emissions.xlsx
 VOC_factors.csv
 VOC_reducido.xlsx


In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

In [2]:
def copert5(alpha, beta, gamma, delta, epsilon, zeta, hta, vm, redFactor):
  if epsilon*vm**2+zeta*vm+hta == 0:
    return 0
  else:
    return (1-redFactor)*(alpha*vm**2 + beta*vm + gamma + delta/vm)/(epsilon * vm**2 + zeta*vm+ hta)

In [8]:
#cargando. excels
#xls = pd.ExcelFile('Copert5.xlsx')
xls = pd.ExcelFile('1.A.3.b.i-iv Road Transport Appendix 4 Emission Factors 2021.xlsx')
df = pd.read_excel(xls,'HOT_EMISSIONS_PARAMETERS', index_col=None)

In [9]:
#Filtrando usando REGEX a las categorías vehiculares a utilizar
df = df.loc[
      ((df['Fuel'] == 'Petrol') |  (df['Fuel'] == 'Diesel')) &
      df.Segment.str.contains('^Medium') | #VLP, TAX
      df.Segment.str.contains("^N1-II$")  | #VLC
      df.Segment.str.contains('^Art*|^Rigid 28 *|^Rigid >32*') | #CAM PES RIGID>28 t and articulated
      df.Segment.str.contains('^Rigid 14 *|^Rigid 20 *|^Rigid 26 *') |#CAM MED RIGID 14-28 t
      df.Segment.str.contains('^Rigid <=7,5 *|^Rigid 7,5 -*|^Rigid 12 *') |#CAM LIV
      df.Segment.str.contains('^Urban Buses Standard 15 *') | #Bus RIG
      df.Segment.str.contains('^Urban Buses Articulated >18 t') | #BUS ART
      df.Segment.str.contains('^Coaches Standard <=18 t') | #BUS INT PROV
      df.Segment.str.contains("^Motorcycles 2-stroke >50 *") #MOT
]


In [10]:
#Cambiando a la clasificacion a utilizar
df.loc[df['Segment'] == 'Medium', 'Segment'] = 'VLP'
df.loc[df['Segment'] == 'N1-II', 'Segment'] = 'VLC'
df.loc[df.Segment.str.contains('^Rigid <=7,5 *|^Rigid 7,5 -*|^Rigid 12 *'), 'Segment'] = 'CAM LIV'
df.loc[df.Segment.str.contains('^Rigid 14 *|^Rigid 20 *|^Rigid 26 *'), 'Segment'] = 'CAM MED'
df.loc[df.Segment.str.contains('^Art*|^Rigid 28 *|^Rigid >32*'), 'Segment'] = 'CAM PES'
df.loc[df.Segment.str.contains('^Urban Buses Standard 15 *'), 'Segment'] = 'BUS RIG'
df.loc[df.Segment.str.contains('^Urban Buses Articulated >18 t'), 'Segment'] = 'BUS ART'
df.loc[df.Segment.str.contains('^Coaches Standard <=18 t'), 'Segment'] = 'BUS INT PROV'
df.loc[df.Segment.str.contains("^Motorcycles 2-stroke >50 *"), 'Segment'] = 'MOT'
#df.to_excel('C5_reducido.xlsx',index = False)
#df.loc[df.Segment.str.contains('^CAM MED*')] 

In [11]:
#df = pd.read_excel('C5_reducido_prueba.xlsx')
df = df.loc[ ((df['Load'].isnull())) | #todas las pendientes y las que no tienen pendiente
       ((df['Load'] == 0.5))] #carga 50% y las que su carga no está definida
#df.to_excel('C5_reducido_prueba.xlsx',index = False)
pd.unique(df["Load"])
df = df.loc[(df["Mode"] == "Highway") | (df["Mode"].isnull()) | (df["Mode"] == "Urban Peak") | (df["Mode"] == "Hot")]
df["Mode"].unique()

array([nan, 'Urban Peak', 'Highway', 'Hot'], dtype=object)

In [287]:
#Seteando velocidad, cada tipo de veh. tiene una velocidad máxima y mínima
# BUS INT PROV + CAM LIV/MED velocidad max 90
# CAM PES velocidad max 80
velMin = 25
velMax = 100

df["velMin"] = np.where(df["Min Speed [km/h]"] > velMin, #verificando si es menor que el valor min
                       df["Min Speed [km/h]"], velMin)
df["velMax"] = np.where((df["Segment"] =="CAM PES"),80, velMax) #seteando velmax como 80 para los CAM PES
df["velMax"] = np.where((df["Segment"] == "BUS INT PROV") | (df["Segment"] == "CAM LIV") | (df["Segment"] ==  "CAM MED"),
                       90, df["velMax"]) 
df["velMax"] = np.where(df["Max Speed [km/h]"] < df["velMax"], #verificando si es mayor que el valor max
                       df["Max Speed [km/h]"], df["velMax"])

#df.to_excel("revisar.xlsx", index=False)
#df.loc[df["Segment"] == "CAM PES"]["velMax"]

In [13]:
#Cambiando clasificaciones
df.loc[df['Euro Standard'] == 'PRE ECE', 'Euro Standard'] = 'No cat'
df.loc[df['Euro Standard'] == 'Euro 1', 'Euro Standard'] = 'E1'
df.loc[df['Euro Standard'] == 'Euro 2', 'Euro Standard'] = 'E2'
df.loc[df['Euro Standard'] == 'Euro 3', 'Euro Standard'] = 'E3'
df.loc[df['Euro Standard'] == 'Euro 4', 'Euro Standard'] = 'E4'
df.loc[df['Euro Standard'] == 'Euro 5', 'Euro Standard'] = 'E5'
df.loc[df['Euro Standard'] == 'Euro 6 a/b/c', 'Euro Standard'] = 'E6abc'
df.loc[df['Euro Standard'] == 'Euro 6 d', 'Euro Standard'] = 'E6d'
df.loc[df['Euro Standard'] == 'Conventional', 'Euro Standard'] = 'No cat'
df.loc[df['Euro Standard'] == 'Euro I', 'Euro Standard'] = 'E1'
df.loc[df['Euro Standard'] == 'Euro II', 'Euro Standard'] = 'E2'
df.loc[df['Euro Standard'] == 'Euro III', 'Euro Standard'] = 'E3'
df.loc[df['Euro Standard'] == 'Euro IV', 'Euro Standard'] = 'E4'
df.loc[df['Euro Standard'] == 'Euro V', 'Euro Standard'] = 'E5'
df.loc[df['Euro Standard'] == 'Euro VI A/B/C', 'Euro Standard'] = 'E6abc' #al parecer no hay diferencia en los parámetros estudiados
df.loc[df['Euro Standard'] == 'Euro VI D/E', 'Euro Standard'] = 'E6d'
df.loc[df['Fuel'] == 'Petrol', 'Fuel'] = 'Gasolina'
#df.to_excel('C5_reducido_prueba.xlsx',index = False)

In [56]:
df = df.loc[
    (df["Euro Standard"] == "No cat") |
    (df["Euro Standard"] == "E1") |
    (df["Euro Standard"] == "E2") |
    (df["Euro Standard"] == "E3") |
    (df["Euro Standard"] == "E4") |
    (df["Euro Standard"] == "E5") |
    (df["Euro Standard"] == "E6abc") |
    (df["Euro Standard"] == "E6d")
]


In [57]:
df = df.loc[
            #Todo lo en blanco
            (df['Technology'].isnull()) |
            #Passenger Cars
            ((df['Category'] == 'Passenger Cars') & 
             (df['Technology'] == 'PFI') | (df['Technology'] == 'DPF')) |
            #Light Commercial Vehicles
            ((df['Category'] == 'Light Commercial Vehicles') & 
             (df['Technology'] == 'PFI') | (df['Technology'] == 'DPF')) |
             #Buses
            ((df['Category'] == 'Buses') & 
             (df['Technology'] == 'EGR') | (df['Technology'] == 'DPF+SCR')) |
            #Heavy Duty Trucks
            ((df['Category'] == 'Heavy Duty Trucks') & 
             (df['Technology'] == 'EGR') | (df['Technology'] == 'DPF+SCR'))
            ]

In [58]:
alpha = df[["Alpha"]].astype("float")
beta = df[["Beta"]].astype("float")
gamma = df[["Gamma"]].astype("float")
delta = df[["Delta"]].astype("float")
epsilon = df[["Epsilon"]].astype("float")
zeta = df[["Zita"]].astype("float")
hta = df[["Hta"]].astype("float")
vmUrb = df[["velMin"]].astype("float")
vmInt = df[["velMax"]].astype("float")
redFactor = df[["Reduction Factor [%]"]].astype("float")
#copert5(alpha, beta, gamma, delta, epsilon, zeta, hta, vm, redFactor)
discriminante = (epsilon.values*vmUrb.values**2+zeta.values*vmUrb.values+
                 hta.values)
df["EF[g/km]_ECF[MJ/km]_Urb"] = np.where(discriminante < 0, 0, (1 - redFactor.values)*((alpha.values * vmUrb.values**2 + beta.values * vmUrb.values 
+ gamma.values + (delta.values/vmUrb.values)) / (discriminante)))
discriminante = (epsilon.values*vmInt.values**2+zeta.values*vmInt.values+
                 hta.values)
df["EF[g/km]_ECF[MJ/km]_Int"] = np.where(discriminante < 0, 0, (1 - redFactor.values)*((alpha.values * vmInt.values**2 + beta.values * vmInt.values 
+ gamma.values + (delta.values/vmInt.values)) / (discriminante)))

In [102]:
cols = list(df.columns)
#df = df[cols[0:9]+cols[11:19]]
dfCO = df.loc[df['Pollutant'] == 'CO']
dfNOx = df.loc[df['Pollutant'] == 'NOx']
dfVOC = df.loc[df['Pollutant'] == 'VOC']
dfPM = df.loc[df['Pollutant'] == 'PM Exhaust']
dfCH4 = df.loc[df['Pollutant'] == 'CH4']
dfCO2 = df.loc[df['Pollutant'] == 'EC']
dfN2O = df.loc[df['Pollutant'] == 'N2O']
dfNH3 = df.loc[df['Pollutant'] == 'NH3']

In [202]:
#Calculo CO2 y SOx
gasDensidad = 0.730 #Ton/m3
gasPcalor = 11200 #Kcal/kg 
dieselPcalor = 10900 #Kcal/kg 
dieselDensidad = 0.840 #ton/m3
tJouleAtCal = 2.39*10**(-1) #de joule a cal 
kCalLt = (gasDensidad*1000*gasPcalor*0.001)
kCalLt = (dieselDensidad*1000*dieselPcalor*0.001)
######## Fin Datos ##########
####### Urbano #######

FCgas_urb = (1000*dfCO2[["EF[g/km]_ECF[MJ/km]_Urb"]]*tJouleAtCal)/(gasPcalor/1000) #gr/km
FCdies_urb = (1000*dfCO2[["EF[g/km]_ECF[MJ/km]_Urb"]]*tJouleAtCal)/(dieselPcalor/1000) #gr/km
FC_urb = np.where(dfCO2[["Fuel"]]=="Gasolina", FCgas_urb, FCdies_urb) #diferenciando diesel y gasolina gr/km
co2Calc_urb = np.where(dfCO2[["Fuel"]] == "Gasolina",44.0011*(FC_urb/(12.011+1.008*1.8)), 44.0011*(FC_urb/(12.011+1.008*2)))
soxCalc_urb = 2*5*(10**-5)*FC_urb
dfCO2[["km/lt_urb"]] = np.where(dfCO2[["Fuel"]]=="Gasolina", (FC_urb/(gasDensidad*10**3))**-1,(FC_urb/(dieselDensidad*10**3))**-1)
dfCO2[["CO2 g/km_urb"]] = co2Calc_urb
dfCO2[["SOx g/km_urb"]] = soxCalc_urb
#### Interurbano ####
FCgas_int = (1000*dfCO2[["EF[g/km]_ECF[MJ/km]_Int"]]*tJouleAtCal)/(gasPcalor/1000) #gr/km
FCdies_int = (1000*dfCO2[["EF[g/km]_ECF[MJ/km]_Int"]]*tJouleAtCal)/(dieselPcalor/1000) #gr/km
FC_int = np.where(dfCO2[["Fuel"]]=="Gasolina", FCgas_int, FCdies_int) #diferenciando diesel y gasolina gr/km
co2Calc_int = np.where(dfCO2[["Fuel"]] == "Gasolina",44.0011*(FC_int/(12.011+1.008*1.8)), 44.0011*(FC_int/(12.011+1.008*2)))
soxCalc_int = 2*5*(10**-5)*FC_int
dfCO2[["km/lt_int"]] = np.where(dfCO2[["Fuel"]]=="Gasolina", (FC_int/(gasDensidad*10**3))**-1,(FC_int/(dieselDensidad*10**3))**-1)
dfCO2[["CO2 g/km_int"]] = co2Calc_int
dfCO2[["SOx g/km_int"]] = soxCalc_int



<ipython-input-202-3d03ae2ad225>:17: RuntimeWarning: divide by zero encountered in reciprocal
  dfCO2[["km/lt_urb"]] = np.where(dfCO2[["Fuel"]]=="Gasolina", (FC_urb/(gasDensidad*10**3))**-1,(FC_urb/(dieselDensidad*10**3))**-1)
<ipython-input-202-3d03ae2ad225>:26: RuntimeWarning: divide by zero encountered in reciprocal
  dfCO2[["km/lt_int"]] = np.where(dfCO2[["Fuel"]]=="Gasolina", (FC_int/(gasDensidad*10**3))**-1,(FC_int/(dieselDensidad*10**3))**-1)


,Category,Fuel,Segment,Euro Standard,Technology,Pollutant,Mode,Road Slope,Load,Min Speed [km/h],...,velMin,velMax,EF[g/km]_ECF[MJ/km]_Urb,EF[g/km]_ECF[MJ/km]_Int,km/lt_urb,CO2 g/km_urb,SOx g/km_urb,km/lt_int,CO2 g/km_int,SOx g/km_int
283,Passenger Cars,Gasolina,VLP,No cat,NaN,EC,NaN,NaN,NaN,10,...,25,80,4.581567,2.928442,7.466704,311.157123,0.009777,11.681707,198.885165,0.006249
339,Passenger Cars,Gasolina,VLP,E1,NaN,EC,NaN,NaN,NaN,5,...,25,80,3.148731,2.119259,10.864442,213.846064,0.006719,16.142058,143.929493,0.004522
347,Passenger Cars,Gasolina,VLP,E2,NaN,EC,NaN,NaN,NaN,5,...,25,80,3.124768,2.071954,10.947757,212.218638,0.006668,16.510604,140.716724,0.004421
363,Passenger Cars,Gasolina,VLP,E3,PFI,EC,NaN,NaN,NaN,5,...,25,80,3.201468,2.187314,10.685473,217.427725,0.006832,15.639822,148.551445,0.004668
379,Passenger Cars,Gasolina,VLP,E4,PFI,EC,NaN,NaN,NaN,5,...,25,80,3.222993,2.312745,10.614112,218.889548,0.006878,14.791600,157.070109,0.004935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23263,L-Category,Gasolina,MOT,E1,NaN,EC,NaN,NaN,NaN,10,...,25,80,1.389802,1.252546,24.614451,94.388382,0.002966,27.311729,85.066683,0.002673
23279,L-Category,Gasolina,MOT,E2,NaN,EC,NaN,NaN,NaN,10,...,25,80,1.389802,1.252546,24.614451,94.388382,0.002966,27.311729,85.066683,0.002673
23295,L-Category,Gasolina,MOT,E3,NaN,EC,NaN,NaN,NaN,10,...,25,80,0.000000,0.000000,inf,0.000000,0.000000,inf,0.000000,0.000000
23311,L-Category,Gasolina,MOT,E4,NaN,EC,NaN,NaN,NaN,10,...,25,80,0.000000,0.000000,inf,0.000000,0.000000,inf,0.000000,0.000000


In [521]:
#Una vez reducida a lo que se utilizara, se separan dataframes diferentes por contaminante.
#df = pd.read_excel('C5_reducido_prueba.xlsx')
dfCO.to_excel('CO_reducido.xlsx',index = False)
dfNOx.to_excel('NOx_reducido.xlsx',index = False)
dfVOC.to_excel('VOC_reducido.xlsx',index = False)
dfPM.to_excel('PM_reducido.xlsx',index = False)
dfCH4.to_excel('CH4_reducido.xlsx',index = False)
dfCO2.to_excel('CO2_reducido.xlsx',index = False)

In [178]:
def formater(gf,cont):
    gf["Gas"] = cont
    gf["Unidad"] = "[g/km]"
    urb_gf = gf.iloc[:,[0,1,2,22,24,25]]
    int_gf = gf.iloc[:,[0,1,2,23,24,25]]
    urb_gf["Ambito"] = "Urbano"
    int_gf["Ambito"] = "Interurbano"
    urb_gf.columns = ["Modo","Motorizacion", "Norma", "2014", "Gas", "Unidad", "Ambito"]
    urb_gf = urb_gf[["Ambito", "Modo", "Motorizacion", "Norma", "Gas", "Unidad", "2014"]]
    int_gf.columns = ["Modo","Motorizacion", "Norma", "2014", "Gas", "Unidad", "Ambito"]
    int_gf = int_gf[["Ambito", "Modo", "Motorizacion", "Norma", "Gas", "Unidad", "2014"]]
    gf2 = pd.concat([urb_gf,int_gf], axis=0)
    new_col = "R1"
    gf2.insert(loc=0, column="Region", value=new_col)
    final=gf2.copy()
    for i in range(2,17):
        new_col = "R"+str(i)
        gf2["Region"]=new_col
        final = pd.concat([final,gf2],axis=0)
    return final

In [311]:
def formater2(gf, cont):
    gf["Gas"] = cont
    gf["Unidad"] = "[g/km]"
    gf_urb = gf.iloc[:,[0,1,2,3,5,6]]
    gf_int = gf.iloc[:,[0,1,2,4,5,6]]
    gf_urb["Ambito"] = "Urbano"
    gf_int["Ambito"] = "Interurbano"
    gf_urb.columns = ["Modo","Motorizacion", "Norma", "2014", "Gas", "Unidad", "Ambito"]
    gf_urb = gf_urb[["Ambito", "Modo", "Motorizacion", "Norma", "Gas", "Unidad", "2014"]]
    gf_int.columns = ["Modo","Motorizacion", "Norma", "2014", "Gas", "Unidad", "Ambito"]
    gf_int = gf_int[["Ambito", "Modo", "Motorizacion", "Norma", "Gas", "Unidad", "2014"]]
    gf2 = pd.concat([gf_urb,gf_int], axis=0)
    new_col = "R1"
    gf2.insert(loc=0, column="Region", value=new_col)
    final=gf2.copy()
    for i in range(2,17):
        new_col = "R"+str(i)
        gf2["Region"]=new_col
        final = pd.concat([final,gf2],axis=0)
    return final

In [204]:
### columns_text = ["Region", "Ambito", "Modo", "Motorizacion", "Norma", "Gas", "Unidad"]
#dfNOx[(dfNOx["Segment"]=="BUS ART") & (dfNOx["Euro Standard"]=="E1")]["EF[g/km]_ECF[MJ/km]_Urb"].mean()
gfNOx = dfNOx.groupby(["Segment","Fuel","Euro Standard"],as_index=False).mean()
gfCO = dfCO.groupby(["Segment","Fuel","Euro Standard"],as_index=False).mean()
gfVOC = dfVOC.groupby(["Segment","Fuel","Euro Standard"],as_index=False).mean()
gfPM = dfPM.groupby(["Segment","Fuel","Euro Standard"],as_index=False).mean()
gfCH4 = dfCH4.groupby(["Segment","Fuel","Euro Standard"],as_index=False).mean()
#formater(gfNOx, "NOx").to_excel("NOx.xlsx", index=False)
#formater(gfCO, "CO").to_excel("CO.xlsx", index=False)
#formater(gfVOC, "VOC").to_excel("VOC.xlsx", index=False)
#formater(gfPM, "PM25").to_excel("PM25.xlsx", index=False)
#formater(gfCH4, "CH4").to_excel("CH4.xlsx", index=False)
gfNOx = formater(gfNOx, "NOx")
gfCO = formater(gfCO, "CO")
gfVOC = formater(gfVOC, "VOC")
gfCH4 = formater(gfCH4, "CH4")
gfPM = formater(gfPM, "PM25")
#pd.concat([gfNOx,gfCO,gfCH4,gfVOC, gfPM], axis=0).to_excel("FE.xlsx", index=False)

In [319]:
gk = df.groupby(["Segment", "Pollutant", "Fuel", 'Euro Standard'],as_index=False).mean()
gk[["EF[g/km]_ECF[MJ/km]_Urb","EF[g/km]_ECF[MJ/km]_Int", "velMax", "velMin"]].to_excel("Resumen_categoria_combustible.xlsx",index = True)
#df.to_excel('C5_reducido_prueba.xlsx',index = False)
#df2 = pd.read_excel('PM_reducido.xlsx')
gk2 = dfCO2.groupby(["Segment", "Pollutant", "Fuel", 'Euro Standard'],as_index=False).mean()
#gk2.to_excel("ResumenCO2.xlsx", index=True)
#gk[["velMin", "velMax", "EF[g/km]_ECF[MJ/km]_Urb", "EF[g/km]_ECF[MJ/km]_Int"]]

kmLt = gk2.iloc[:,[0,2,3,25,28]]
emCO2 = gk2.iloc[:,[0,2,3,26,29]]
emSOx = gk2.iloc[:,[0,2,3,27,30]]
gfSOx = formater2(emSOx, "SOx")
gfCO2 = formater2(emCO2, "CO2")
gfkmLt = formater2(kmLt, "km/lt")
#(df[df["Segment"]=="CAM PES"]["Max Speed [km/h]"])
gf_concat = pd.concat([gfNOx,gfCO,gfCH4,gfVOC, gfPM, gfSOx, gfCO2, gfkmLt], axis=0)
#final.to_excel("NOx.xlsx", index=False)
n = 2060-2014
years = (pd.concat([gf_concat[["2014"]]] * n, axis=1, ignore_index=True)).rename(lambda x: str(2015 + x), axis=1)
pd.concat([gf_concat,years], axis=1).to_excel("FE.xlsx", index=False)

## Toques finales a CH4, PM exhaust
Archivos = ['CH4_reducido.xlsx','PM_reducido.xlsx']
for file in Archivos:
  df = pd.read_excel(file)
  df = df.loc[
              df['Mode'].isnull() |
              (df['Mode'] == 'Highway') |
              (df['Mode'] == 'Urban Off Peak')
              ]
  df.loc[df['Mode'] == 'Highway', 'Mode'] = 'Interurbano'
  df.loc[df['Mode'] == 'Urban Off Peak', 'Mode'] = 'Urbano'
  df.to_excel(file,index = False)

In [7]:
df = pd.read_pickle("emisiones.pkl")
df

,Tipo,Escenario,Region,Modo,Motorizacion,Gas,Año,Emisiones,Ambito
0,Pasajero,Escenario_Tendencial,R1,Bus Rígido,Diesel,BC,2019,1.461921e+10,Urbano
1,Pasajero,Escenario_Tendencial,R1,Bus Rígido,Diesel,BC,2020,1.196844e+10,Urbano
2,Pasajero,Escenario_Tendencial,R1,Bus Rígido,Diesel,BC,2021,1.140410e+10,Urbano
3,Pasajero,Escenario_Tendencial,R1,Bus Rígido,Diesel,BC,2022,1.067120e+10,Urbano
4,Pasajero,Escenario_Tendencial,R1,Bus Rígido,Diesel,BC,2023,1.031615e+09,Urbano
...,...,...,...,...,...,...,...,...,...
148675,Carga,Escenario_Tendencial,RM,Ferroviario,Diesel,VOC,2056,0.000000e+00,Interurbano
148676,Carga,Escenario_Tendencial,RM,Ferroviario,Diesel,VOC,2057,0.000000e+00,Interurbano
148677,Carga,Escenario_Tendencial,RM,Ferroviario,Diesel,VOC,2058,0.000000e+00,Interurbano
148678,Carga,Escenario_Tendencial,RM,Ferroviario,Diesel,VOC,2059,0.000000e+00,Interurbano
